In [1]:
DATA_NAME = 'coco-indoor' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_horizontal_wavelet_indoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,5961.14,0.12,0.04,0.03,0.00,0.00,-0.00,-0.00
3,0.12,1395.29,-0.02,0.01,-0.00,0.00,-0.00,0.00
4,0.04,-0.02,303.51,0.00,0.00,-0.00,0.00,-0.00
5,0.03,0.01,0.00,56.95,0.00,0.00,0.00,-0.00
6,0.00,-0.00,0.00,0.00,10.21,0.00,0.00,0.00
7,0.00,0.00,-0.00,0.00,0.00,1.81,-0.00,0.00
8,-0.00,-0.00,0.00,0.00,0.00,-0.00,0.34,0.00
9,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.07


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,0.00004,0.00003,0.00005,0.00001,0.00000,-0.00000,-0.00002
3,0.00004,1.00000,-0.00002,0.00004,-0.00001,0.00002,-0.00005,0.00009
4,0.00003,-0.00002,1.00000,0.00002,0.00001,-0.00006,0.00007,-0.00006
5,0.00005,0.00004,0.00002,1.00000,0.00002,0.00002,0.00002,-0.00004
6,0.00001,-0.00001,0.00001,0.00002,1.00000,0.00002,0.00003,0.00004
7,0.00000,0.00002,-0.00006,0.00002,0.00002,1.00000,-0.00004,0.00001
8,-0.00000,-0.00005,0.00007,0.00002,0.00003,-0.00004,1.00000,0.00003
9,-0.00002,0.00009,-0.00006,-0.00004,0.00004,0.00001,0.00003,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.18972055365459273

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[4.44734091e+06 2.38452904e+05 1.09675282e+04 3.70915460e+02
 1.11776843e+01 3.15088315e-01 8.72645777e-03 2.25026871e-35]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999351,-0.035267,-0.007212,-0.001346,-0.000242,-0.000043,-0.000008,-0.000002
1,0.034956,0.998632,-0.038262,-0.006786,-0.001216,-0.000215,-0.000042,-0.000008
2,0.008481,0.037685,0.998465,-0.039095,-0.006719,-0.001198,-0.000223,-0.000050
3,0.001897,0.008130,0.038436,0.998114,-0.046428,-0.007927,-0.001496,-0.000328
4,0.000424,0.001814,0.008344,0.045535,0.996983,-0.061226,-0.011103,-0.002418
5,0.000099,0.000424,0.001959,0.010418,0.059367,0.993209,-0.097432,-0.020169
6,0.000027,0.000116,0.000526,0.002810,0.015697,0.090010,0.972121,-0.215925
7,0.000012,0.000052,0.000241,0.001283,0.007152,0.040276,0.212982,0.976198


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0006490285567436471,
 0.0013676683686477675,
 0.001534732667677674,
 0.0018856783948268951,
 0.0030166726865906712,
 0.0067914380719228395,
 0.027878633515846074,
 0.023801509476081395]